In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-dataset/dataset/._sample_test_out.csv
/kaggle/input/amazon-dataset/dataset/._train.csv
/kaggle/input/amazon-dataset/dataset/._sample_test.csv
/kaggle/input/amazon-dataset/dataset/._sample_test_out_fail.csv
/kaggle/input/amazon-dataset/dataset/._test.csv
/kaggle/input/amazon-dataset/src/._constants.py
/kaggle/input/amazon-dataset/src/._utils.py
/kaggle/input/amazon-dataset/src/._sanity.py
/kaggle/input/amazon-dataset/src/._test.ipynb
/kaggle/input/amazon-dataset/student_resource 3/._src
/kaggle/input/amazon-dataset/student_resource 3/._sample_code.py
/kaggle/input/amazon-dataset/student_resource 3/README.md
/kaggle/input/amazon-dataset/student_resource 3/._README.md
/kaggle/input/amazon-dataset/student_resource 3/._dataset
/kaggle/input/amazon-dataset/student_resource 3/sample_code.py
/kaggle/input/amazon-dataset/student_resource 3/dataset/sample_test.csv
/kaggle/input/amazon-dataset/student_resource 3/dataset/._sample_test_out.csv
/kaggle/input/amazon-dataset/stude

In [3]:
df = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [4]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [5]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


# For single image extraction

1) To see detailed extraction

In [223]:
"""import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Define regex patterns based on the allowed units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units set based on entity_unit_map
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Preprocessing function to clean up OCR text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Replace "O" with "0" (to handle common OCR errors like "140OmG")
    text = text.replace("o", "0")
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any unnecessary punctuation that might interfere with matching
    text = re.sub(r'[^\w\s.,]', '', text)
    
    return text

# Function to extract various units from the text
def extract_units(text, entity_name):
    extracted_units = set()

    # Preprocess the text before extracting units
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        unit_list = entity_unit_map[entity_name]
        for unit in unit_list:
            if unit in patterns:
                pattern = patterns[unit]
                matches = re.findall(pattern, cleaned_text)
                if matches:
                    # Format as "value unit" and store unique entries
                    for match in matches:
                        value = float(match[0])
                        formatted_entry = f"{value} {unit}"
                        extracted_units.add(formatted_entry)

    return extracted_units

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to process each image URL
def process_image_url(image_url, entity_name):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units from the final text based on the entity_name
    units = extract_units(final_text, entity_name)
    
    return final_text, best_angle, units

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Limit to first 30 links for processing
df = df.head(30)

# Initialize a list to store results
results = []

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_link'
    entity_name = row['entity_name']  # Assuming this column specifies what entity to extract (e.g., weight, volume)
    
    print(f"Processing image from URL: {image_url} for entity: {entity_name}")
    
    final_text, best_angle, units = process_image_url(image_url, entity_name)
    
    # Prepare the result row
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)  # Convert set to list for the predicted result
    }
    
    # Append the result to the list
    results.append(result_row)
    
    # Print results for each image
    print(f"Extracted Text (Best angle {best_angle} degrees): {final_text}")
    print(f"Extracted Units for {entity_name}: {result_row['predicted']}")
    print("--------------------------------------------------")

# Create a new DataFrame from results and save it as a CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)
"""

'import easyocr\nimport pandas as pd\nfrom PIL import Image\nimport numpy as np\nimport requests\nfrom io import BytesIO\nimport re\n\n# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)\nreader = easyocr.Reader([\'en\'], gpu=True)\n\n# Define regex patterns based on the allowed units\npatterns = {\n    \'gram\': r\'(\\d+(\\.\\d+)?)\\s*(g|grams?)\',\n    \'kilogram\': r\'(\\d+(\\.\\d+)?)\\s*(kg|kilograms?)\',\n    \'milligram\': r\'(\\d+(\\.\\d+)?)\\s*(mg|milligrams?)\',\n    \'microgram\': r\'(\\d+(\\.\\d+)?)\\s*(µg|micrograms?)\',\n    \'ounce\': r\'(\\d+(\\.\\d+)?)\\s*(oz|ounces?)\',\n    \'pound\': r\'(\\d+(\\.\\d+)?)\\s*(lb|pounds?)\',\n    \'ton\': r\'(\\d+(\\.\\d+)?)\\s*(ton|tons?)\',\n    \'centimetre\': r\'(\\d+(\\.\\d+)?)\\s*(cm|centimetre|centimeters?)\',\n    \'foot\': r\'(\\d+(\\.\\d+)?)\\s*(ft|foot|feet)\',\n    \'inch\': r\'(\\d+(\\.\\d+)?)\\s*(in|inch|inches)\',\n    \'metre\': r\'(\\d+(\\.\\d+)?)\\s*(m|metre|meters?)\',\n    \'millimetr

2)Didn't want to see details

In [6]:
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re
import warnings

# Suppress FutureWarnings and other warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Define regex patterns based on the allowed units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units set based on entity_unit_map
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Preprocessing function to clean up OCR text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Replace "O" with "0" (to handle common OCR errors like "140OmG")
    text = text.replace("o", "0")
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any unnecessary punctuation that might interfere with matching
    text = re.sub(r'[^\w\s.,]', '', text)
    
    return text

# Function to extract various units from the text
def extract_units(text, entity_name):
    extracted_units = set()

    # Preprocess the text before extracting units
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        unit_list = entity_unit_map[entity_name]
        for unit in unit_list:
            if unit in patterns:
                pattern = patterns[unit]
                matches = re.findall(pattern, cleaned_text)
                if matches:
                    # Format as "value unit" and store unique entries
                    for match in matches:
                        value = float(match[0])
                        formatted_entry = f"{value} {unit}"
                        extracted_units.add(formatted_entry)

    return extracted_units

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to process each image URL
def process_image_url(image_url, entity_name):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units from the final text based on the entity_name
    units = extract_units(final_text, entity_name)
    
    return final_text, best_angle, units

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Limit to first 50 links for processing
df = df.head(50)

# Initialize a list to store results
results = []

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_link'
    entity_name = row['entity_name']  # Assuming this column specifies what entity to extract (e.g., weight, volume)
    
    # Process image
    final_text, best_angle, units = process_image_url(image_url, entity_name)
    
    # Prepare the result row
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)  # Convert set to list for the predicted result
    }
    
    # Append the result to the list
    results.append(result_row)

# Print the count of processed images
print(f"Processed {len(results)} images.")

# Create a new DataFrame from results and save it as a CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)

Processed 50 images.


In [30]:
pred = pd.read_csv('predicted_units.csv')

In [31]:
new_train = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [32]:
# Merging df1 and test1 on image_link, entity_name, and group_id
pred = pred.merge(new_train, on=['image_link', 'entity_name'], how='inner')

# Optionally, if you need to remove duplicates based on these columns:
pred = pred.drop_duplicates(subset=['image_link', 'entity_name', 'group_id'])

In [33]:
# Convert 'predicted' column from a list of strings to a single string without brackets and quotes
pred['predicted'] = pred['predicted'].apply(lambda x: ', '.join(sorted(set(x.strip('[]').replace("'", "").split(', ')))))

In [51]:
pred = pred[['image_link', 'entity_name', 'extracted_text', 'predicted', 'group_id', 'entity_value']]

In [49]:
import pandas as pd
import re

# Function to convert values to a common unit (e.g., grams) for comparison
def convert_to_grams(value):
    if pd.isna(value):
        return None
    value = value.lower().strip()
    # Extract the numeric part and unit
    match = re.match(r"([\d.]+)\s*(\w+)", value)
    if match:
        number = float(match.group(1))
        unit = match.group(2)
        if unit in ["gram", "grams", "g"]:
            return number
        elif unit in ["milligram", "milligrams", "mg"]:
            return number / 1000
        elif unit in ["kilogram", "kilograms", "kg"]:
            return number * 1000
        elif unit in ["ounce", "ounces", "oz"]:
            return number * 28.3495
        elif unit in ["pound", "pounds", "lb"]:
            return number * 453.592
    return None

# Function to get the highest value in grams and return the original value
def get_highest_value(row):
    predicted_values = row['predicted'].split(',')
    highest_value = None
    original_value = None
    for value in predicted_values:
        value_in_grams = convert_to_grams(value)
        if value_in_grams is not None:
            if highest_value is None or value_in_grams > highest_value:
                highest_value = value_in_grams
                original_value = value.strip()
    return original_value if original_value is not None else ""

# Apply the function to create a new column 'predict'
pred['predict'] = pred.apply(get_highest_value, axis=1)

/tmp/ipykernel_36/1197466700.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred['predict'] = pred.apply(get_highest_value, axis=1)


In [52]:
# Display the cleaned DataFrame
pred.head(30)

,image_link,entity_name,extracted_text,predicted,group_id,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,item_weight,PROPOS' NATUREJ INGREDIENT MENAGER MULTI-USAGE...,500.0 gram,748919,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,item_volume,TLael=_ 767222 Xe RRIFIC; LEBENSMITTELECHT G D...,,916768,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,item_weight,COMPOSITION Serving Size: Tablet (0.709 g) Eac...,"0.04 gram, 0.09 gram, 0.2 gram, 0.51 gram, 0.7...",459516,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,item_weight,conpositioN Serving Size: Tablet (0.709 g1 Eac...,"0.2 gram, 0.51 gram, 0.709 gram, 10.0 milligra...",459516,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,item_weight,Horbaach' HIGH StRENGTH PSYLLIUM HUSK PLANTAGO...,1400.0 milligram,731432,1400 milligram
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,item_weight,"4JEEQJOH sue1ue1aba, 3 suebaA JOJ aqens - buiu...",,731432,1400 milligram
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,item_weight,"HOrbaach Directions: For adults, take two (2) ...",1400.0 milligram,731432,1400 milligram
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,item_weight,VEGAN WHEAT FREE soy FREE OVATA DAIRY FREE FRE...,1400.0 milligram,731432,1400 milligram
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,item_weight,Horbaach V 100% HIGHEST QUALITY TO HGH GMP ) H...,1400.0 milligram,731432,1400 milligram
9,https://m.media-amazon.com/images/I/71jBLhmTNl...,item_weight,NEW LOOK SAME TRUSTEDQUALITY OLD Horbaach NEW ...,"1400.0 milligram, 400.0 milligram",731432,1400 milligram


In [59]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate the prediction and the matching score
def predict_presence(entity_value, predicted_value, threshold=70):
    # Calculate the fuzzy match ratio between the entity and the predicted value
    match_score = fuzz.partial_ratio(entity_value.lower(), predicted_value.lower())
    
    # Predict presence based on the match score exceeding the threshold
    presence = 1 if match_score >= threshold else 0
    
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Function to get the best match from the predicted values
def get_best_prediction(row, threshold=50):
    predictions = row['predicted'].split(", ")  # Split multiple predicted values
    best_prediction = None
    best_probability = 0
    
    for predicted_value in predictions:
        presence, probability = predict_presence(row['entity_value'], predicted_value)
        # Only consider values that meet the probability threshold and are better than the current best match
        if probability > best_probability and probability > (threshold / 100):
            best_prediction = predicted_value
            best_probability = probability
    
    return best_prediction, best_probability

# Apply the function to update the DataFrame
pred[['prediction', 'probability']] = pred.apply(
    lambda row: get_best_prediction(row), axis=1, result_type='expand'
)


In [60]:
# Show the DataFrame with predictions and probabilities
pred.head(30)


,image_link,entity_name,extracted_text,predicted,group_id,entity_value,prediction,probability
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,item_weight,PROPOS' NATUREJ INGREDIENT MENAGER MULTI-USAGE...,500.0 gram,748919,500.0 gram,500.0 gram,1.00
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,item_volume,TLael=_ 767222 Xe RRIFIC; LEBENSMITTELECHT G D...,,916768,1.0 cup,NaN,0.00
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,item_weight,COMPOSITION Serving Size: Tablet (0.709 g) Eac...,"0.04 gram, 0.09 gram, 0.2 gram, 0.51 gram, 0.7...",459516,0.709 gram,0.709 gram,1.00
3,https://m.media-amazon.com/images/I/612mrlqiI4...,item_weight,conpositioN Serving Size: Tablet (0.709 g1 Eac...,"0.2 gram, 0.51 gram, 0.709 gram, 10.0 milligra...",459516,0.709 gram,0.709 gram,1.00
4,https://m.media-amazon.com/images/I/617Tl40LOX...,item_weight,Horbaach' HIGH StRENGTH PSYLLIUM HUSK PLANTAGO...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,item_weight,"4JEEQJOH sue1ue1aba, 3 suebaA JOJ aqens - buiu...",,731432,1400 milligram,NaN,0.00
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,item_weight,"HOrbaach Directions: For adults, take two (2) ...",1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,item_weight,VEGAN WHEAT FREE soy FREE OVATA DAIRY FREE FRE...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,item_weight,Horbaach V 100% HIGHEST QUALITY TO HGH GMP ) H...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
9,https://m.media-amazon.com/images/I/71jBLhmTNl...,item_weight,NEW LOOK SAME TRUSTEDQUALITY OLD Horbaach NEW ...,"1400.0 milligram, 400.0 milligram",731432,1400 milligram,1400.0 milligram,0.86


# Training the model

for black n white conversion 

In [ ]:
"""import requests
from PIL import Image
from io import BytesIO

def download_image_from_url(image_url, save_path):
    try:
        # Send a GET request to the URL
        response = requests.get(image_url)
        
        # Raise an exception if the request was unsuccessful
        response.raise_for_status()
        
        # Open the image from the fetched bytes
        image = Image.open(BytesIO(response.content))
        
        # Convert the image to black and white (grayscale)
        image_bw = image.convert('L')
        
        # Save the black and white image to the specified path
        image_bw.save(save_path)
        
        print(f"Black and white image successfully saved at {save_path}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading the image: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
image_url = 'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'
save_path = 'downloaded_image_bw.jpg'

download_image_from_url(image_url, save_path)
"""

# For multiple images extraction using links in the csv file

In [ ]:
""""
import requests
from PIL import Image
from io import BytesIO
import os

def download_images_from_df(df, save_directory, url_column='image link', limit=1000):
    try:
        # Create the directory to save images if it doesn't exist
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        
        # Ensure the column name is correct
        if url_column not in df.columns:
            print(f"Column '{url_column}' not found. Available columns: {df.columns.tolist()}")
            return
        
        # Counter to track how many images have been downloaded
        download_count = 0
        
        # Loop through the URLs in the specified column, but stop after reaching the limit
        for index, row in df.iterrows():
            if download_count >= limit:
                print(f"Reached the download limit of {limit} images.")
                break
            
            image_url = row[url_column]
            try:
                # Ensure the URL is valid and not NaN or empty
                if pd.isna(image_url) or not isinstance(image_url, str):
                    continue

                # Send a GET request to the URL
                response = requests.get(image_url)
                response.raise_for_status()
                
                # Open the image from the fetched bytes
                image = Image.open(BytesIO(response.content))
                
                # Construct the save path using index to make each filename unique
                image_save_path = os.path.join(save_directory, f'image_{index + 1}.jpg')
                
                # Save the image
                image.save(image_save_path)
                
                # Increment the download count and show only the count
                download_count += 1
                print(f"Total downloaded: {download_count}", end='\r')  # Updates the same line
                
            except requests.exceptions.RequestException:
                continue
            except Exception:
                continue
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
save_directory = 'downloaded_images'  # Directory to save downloaded images

# Assuming your DataFrame is already loaded in 'df'
# Control the limit of downloads here
download_images_from_df(df, save_directory, url_column='image_link', limit=10)
"""

In [ ]:
""""
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd

def download_images_from_df(df, save_directory, url_column='image link', limit=1000):
    try:
        # Create the directory to save images if it doesn't exist
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        
        # Ensure the column name is correct
        if url_column not in df.columns:
            print(f"Column '{url_column}' not found. Available columns: {df.columns.tolist()}")
            return
        
        # Counter to track how many images have been downloaded
        download_count = 0
        
        # Loop through the URLs in the specified column, but stop after reaching the limit
        for index, row in df.iterrows():
            if download_count >= limit:
                print(f"Reached the download limit of {limit} images.")
                break
            
            image_url = row[url_column]
            try:
                # Ensure the URL is valid and not NaN or empty
                if pd.isna(image_url) or not isinstance(image_url, str):
                    continue

                # Send a GET request to the URL
                response = requests.get(image_url)
                response.raise_for_status()
                
                # Open the image from the fetched bytes
                image = Image.open(BytesIO(response.content))
                
                # Construct the save path using index to make each filename unique
                image_save_path = os.path.join(save_directory, f'image_{index + 1}.jpg')
                
                # Save the image
                image.save(image_save_path)
                
                # Increment the download count and show only the count
                download_count += 1
                print(f"Total downloaded: {download_count}", end='\r')  # Updates the same line
                
            except requests.exceptions.RequestException:
                continue
            except Exception:
                continue
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
save_directory = 'downloaded_images'  # Directory to save downloaded images

# Assuming your DataFrame is already loaded in 'df'
# Control the limit of downloads here
download_images_from_df(df, save_directory, url_column='image_link', limit=100)
"""

# Test image extraction

In [ ]:
"""
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd

def download_images_from_df(df, save_directory, url_column='image link', limit=1000):
    try:
        # Create the directory to save images if it doesn't exist
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)
        
        # Ensure the column name is correct
        if url_column not in df.columns:
            print(f"Column '{url_column}' not found. Available columns: {df.columns.tolist()}")
            return
        
        # Counter to track how many images have been downloaded
        download_count = 0
        
        # Loop through the URLs in the specified column, but stop after reaching the limit
        for index, row in df.iterrows():
            if download_count >= limit:
                print(f"Reached the download limit of {limit} images.")
                break
            
            image_url = row[url_column]
            try:
                # Ensure the URL is valid and not NaN or empty
                if pd.isna(image_url) or not isinstance(image_url, str):
                    continue

                # Send a GET request to the URL
                response = requests.get(image_url)
                response.raise_for_status()
                
                # Open the image from the fetched bytes
                image = Image.open(BytesIO(response.content))
                
                # Construct the save path using index to make each filename unique
                image_save_path = os.path.join(save_directory, f'image_{index + 1}.jpg')
                
                # Save the image
                image.save(image_save_path)
                
                # Increment the download count and show only the count
                download_count += 1
                print(f"Total downloaded: {download_count}", end='\r')  # Updates the same line
                
            except requests.exceptions.RequestException:
                continue
            except Exception:
                continue
    
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
save_directory = 'downloaded_test_images'  # Directory to save downloaded images

# Assuming your DataFrame is already loaded in 'df'
# Control the limit of downloads here
download_images_from_df(df, save_directory, url_column='image_link', limit=100)
"""

# Single image text extraction

In [ ]:
"""
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to check if the text contains weight in grams and extract the value
def extract_weight_in_grams(text):
    # Regular expression pattern to match weight values in grams (e.g., 100g, 250 grams, 0.709g)
    weight_pattern = r'(\d+(\.\d+)?)\s*(g|grams)'
    
    # Search the text for the pattern
    match = re.search(weight_pattern, text.lower())  # Convert text to lowercase for better matching
    
    if match:
        # If a match is found, extract the numerical part (group 1) and return it as a float
        return float(match.group(1))
    else:
        return "Weight in grams not found"

# Function to process each image URL
def process_image_url(image_url):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract weight in grams from the final text
    weight = extract_weight_in_grams(final_text)
    
    return final_text, best_angle, weight

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_url'
    print(f"Processing image from URL: {image_url}")
    
    final_text, best_angle, weight = process_image_url(image_url)
    
    # Print results for each image
    print(f"Extracted Text (Best angle {best_angle} degrees): {final_text}")
    print(f"Detected weight in grams: {weight} grams")
    print("--------------------------------------------------")
"""

In [ ]:
"""
import easyocr
from PIL import Image
import re
import numpy as np

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to check if the text contains weight in grams and extract the value
def extract_weight_in_grams(text):
    # Regular expression pattern to match weight values in grams (e.g., 100g, 250 grams, 0.709g)
    weight_pattern = r'(\d+(\.\d+)?)\s*(g|grams)'
    
    # Search the text for the pattern
    match = re.search(weight_pattern, text.lower())  # Convert text to lowercase for better matching
    
    if match:
        # If a match is found, extract the numerical part (group 1) and return it as a float
        return float(match.group(1))
    else:
        return "Weight in grams not found"

# Example of using the provided black-and-white image directly
# Load the image (assuming it's already black-and-white)
image = Image.open('/kaggle/working/downloaded_image_bw.jpg')

# Rotate image if needed and get the corrected text
final_text, best_angle = rotate_and_extract(image)

# Print the extracted text and best angle
print(f"Extracted Text (Best angle {best_angle} degrees):", final_text)

# Extract weight in grams from the final text
weight = extract_weight_in_grams(final_text)

# Output the result
print("Detected weight in grams:", weight, "grams")
"""

# Image folder text extraction

In [ ]:
"""
import easyocr
from PIL import Image
import re
import os

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=False)

# Function to rotate the image and check OCR text
def rotate_and_extract(image_path):
    # Open the image
    img = Image.open(image_path)
    
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = img.rotate(angle, expand=True)
        rotated_img_path = f"/kaggle/working/downloaded_images2/rotated_{angle}.jpg"
        rotated_img.save(rotated_img_path)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_path)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to check if the text contains weight in grams and extract the value
def extract_weight_in_grams(text):
    # Regular expression pattern to match weight values in grams (e.g., 100g, 250 grams, 0.709g)
    weight_pattern = r'(\d+(\.\d+)?)\s*(g|grams)'
    
    # Search the text for the pattern
    match = re.search(weight_pattern, text.lower())  # Convert text to lowercase for better matching
    
    if match:
        # If a match is found, extract the numerical part (group 1) and return it as a float
        return float(match.group(1))
    else:
        return "Weight in grams not found"

# Directory containing images
image_directory = '/kaggle/working/downloaded_images2/'

# List all image files in the directory
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

# Process each image file
for image_file in image_files:
    image_path = os.path.join(image_direcatory, image_file)
    
    # Rotate image if needed and get the corrected text
    final_text, best_angle = rotate_and_extract(image_path)
    
    # Print the extracted text and best angle
    print(f"\nImage: {image_file}")
    print(f"Extracted Text (Best angle {best_angle} degrees):", final_text)
    
    # Extract weight in grams from the final text
    weight = extract_weight_in_grams(final_text)
    
    # Output the result
    print("Detected weight in grams:", weight, "grams")
"""

In [ ]:
"""import easyocr
from PIL import Image
import re
import os

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=False)

# Function to rotate the image and check OCR text
def rotate_and_extract(image_path):
    # Open the image
    img = Image.open(image_path)
    
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = img.rotate(angle, expand=True)
        rotated_img_path = f"/kaggle/working/downloaded_images2/rotated_{angle}.jpg"
        rotated_img.save(rotated_img_path)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_path)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to extract weight in grams
def extract_weight_in_grams(text):
    weight_pattern = r'(\d+(\.\d+)?)\s*(g|grams)'
    match = re.search(weight_pattern, text.lower())
    if match:
        return float(match.group(1)), "grams"
    else:
        return "Weight in grams not found"

# Function to extract weight in milligrams (mg)
def extract_weight_in_milligrams(text):
    milligram_pattern = r'(\d+(\.\d+)?)\s*(mg|milligrams?)'
    match = re.search(milligram_pattern, text.lower())
    if match:
        return float(match.group(1)), "milligrams"
    else:
        return "Weight in milligrams not found"

# Function to extract weight in kilograms (kg)
def extract_weight_in_kilograms(text):
    kilogram_pattern = r'(\d+(\.\d+)?)\s*(kg|kilograms?)'
    match = re.search(kilogram_pattern, text.lower())
    if match:
        return float(match.group(1)), "kilograms"
    else:
        return "Weight in kilograms not found"

# Function to extract weight in ounces (oz)
def extract_weight_in_ounces(text):
    ounce_pattern = r'(\d+(\.\d+)?)\s*(oz|ounce|ounces?)'
    match = re.search(ounce_pattern, text.lower())
    if match:
        return float(match.group(1)), "ounces"
    else:
        return "Weight in ounces not found"

# Function to extract force in newtons (N)
def extract_force_in_newtons(text):
    newton_pattern = r'(\d+(\.\d+)?)\s*(n|newtons?)'
    match = re.search(newton_pattern, text.lower())
    if match:
        return float(match.group(1)), "newtons"
    else:
        return "Force in newtons not found"

# Function to extract volume in liters/milliliters
def extract_volume_in_liters(text):
    volume_pattern = r'(\d+(\.\d+)?)\s*(l|liter|liters|ml|milliliters|milliliter)'
    match = re.search(volume_pattern, text.lower())
    if match:
        value = float(match.group(1))
        unit = match.group(3)
        if unit in ['ml', 'milliliters', 'milliliter']:
            value /= 1000.0  # Convert milliliters to liters
        return value, "liters"
    else:
        return "Volume in liters/milliliters not found"

# Function to extract wattage in W
def extract_wattage(text):
    wattage_pattern = r'(\d+(\.\d+)?)\s*(w|watt|watts)'
    match = re.search(wattage_pattern, text.lower())
    if match:
        return float(match.group(1)), "watts"
    else:
        return "Wattage not found"

# Function to extract voltage in V
def extract_voltage(text):
    voltage_pattern = r'(\d+(\.\d+)?)\s*(v|volt|volts|voltage)'
    match = re.search(voltage_pattern, text.lower())
    if match:
        return float(match.group(1)), "volts"
    else:
        return "Voltage not found"

# Function to extract length in inches, cm, meters, or feet
def extract_length(text):
    length_pattern = r'(\d+(\.\d+)?)\s*(in|inch|cm|centimeter|meters?|m|ft|foot|feet)'
    match = re.search(length_pattern, text.lower())
    if match:
        value = float(match.group(1))
        unit = match.group(3)
        return value, unit
    else:
        return "Length not found"

# Function to extract weight in pounds (lbs)
def extract_weight_in_lbs(text):
    pound_pattern = r'(\d+(\.\d+)?)\s*(lbs|pounds?)'
    match = re.search(pound_pattern, text.lower())
    if match:
        return float(match.group(1)), "lbs"
    else:
        return "Weight in pounds not found"

# Directory containing images
image_directory = '/kaggle/working/downloaded_images2/'

# List all image files in the directory
image_files = [f for f in os.listdir(image_directory) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

# Process each image file
for image_file in image_files:
    image_path = os.path.join(image_directory, image_file)
    
    # Rotate image if needed and get the corrected text
    final_text, best_angle = rotate_and_extract(image_path)
    
    # Print the extracted text and best angle
    print(f"\nImage: {image_file}")
    print(f"Extracted Text (Best angle {best_angle} degrees):", final_text)
    
    # Extract weight in grams
    weight = extract_weight_in_grams(final_text)
    print("Detected weight in grams:", weight)
    
    # Extract weight in milligrams
    weight_mg = extract_weight_in_milligrams(final_text)
    print("Detected weight in milligrams:", weight_mg)
    
    # Extract weight in kilograms
    weight_kg = extract_weight_in_kilograms(final_text)
    print("Detected weight in kilograms:", weight_kg)
    
    # Extract weight in ounces
    weight_oz = extract_weight_in_ounces(final_text)
    print("Detected weight in ounces:", weight_oz)
    
    # Extract force in newtons
    force_n = extract_force_in_newtons(final_text)
    print("Detected force in newtons:", force_n)
    
    # Extract volume in liters
    volume = extract_volume_in_liters(final_text)
    if isinstance(volume, tuple):
        print(f"Detected volume: {volume[0]} {volume[1]}")
    else:
        print(volume)
    
    # Extract wattage
    wattage = extract_wattage(final_text)
    if isinstance(wattage, tuple):
        print(f"Detected wattage: {wattage[0]} {wattage[1]}")
    else:
        print(wattage)
    
    # Extract voltage
    voltage = extract_voltage(final_text)
    if isinstance(voltage, tuple):
        print(f"Detected voltage: {voltage[0]} {voltage[1]}")
    else:
        print(voltage)
    
    # Extract length
    length = extract_length(final_text)
    if isinstance(length, tuple):
        print(f"Detected length: {length[0]} {length[1]}")
    else:
        print(length)
    
    # Extract weight in pounds (lbs)
    weight_lbs = extract_weight_in_lbs(final_text)
    if isinstance(weight_lbs, tuple):
        print(f"Detected weight in pounds: {weight_lbs[0]} {weight_lbs[1]}")
    else:
        print(weight_lbs)
"""

# Text extraction for training

In [ ]:
"""
import easyocr
from PIL import Image
import re
import os
import pandas as pd
import requests
from io import BytesIO
import concurrent.futures
import numpy as np

# Initialize the EasyOCR reader (using GPU if available)
reader = easyocr.Reader(['en'], gpu=True)

# Function to download an image or load it from the local directory
def download_image(image_url, save_directory):
    try:
        response = requests.get(image_url, timeout=5)
        img = Image.open(BytesIO(response.content))
        image_name = os.path.basename(image_url)
        save_path = os.path.join(save_directory, image_name)
        img.save(save_path)
        return save_path
    except Exception as e:
        print(f"Error downloading image {image_url}: {e}")
        return None

# Function to extract text from an image without rotation
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)

        # Resize image to a smaller size for faster processing (reduce by half)
        img = img.resize((img.width // 2, img.height // 2))

        # Convert image to NumPy array for OCR
        img_np = np.array(img)

        # Extract text using OCR
        img_text = reader.readtext(img_np)
        final_text = " ".join([text for _, text, __ in img_text])

        return final_text
    except OSError as e:
        print(f"Error processing image {image_path}: {e}")
        return ""

# Function to search for a specific entity using regular expressions
def search_entity(text, entity):
    patterns = {
        'grams': r'(\d+(\.\d+)?)\s*(g|grams?)',
        'milligrams': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
        'kilograms': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
        'ounces': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
        'newtons': r'(\d+(\.\d+)?)\s*(n|newtons?)',
        'liters': r'(\d+(\.\d+)?)\s*(l|liters?|ml|milliliters?)',
        'watts': r'(\d+(\.\d+)?)\s*(w|watts?)',
        'volts': r'(\d+(\.\d+)?)\s*(v|volts?)',
        'inches': r'(\d+(\.\d+)?)\s*(in|inches?)',
        'centimeters': r'(\d+(\.\d+)?)\s*(cm|centimeters?)',
        'meters': r'(\d+(\.\d+)?)\s*(m|meters?)',
        'feet': r'(\d+(\.\d+)?)\s*(ft|feet?)',
        'pounds': r'(\d+(\.\d+)?)\s*(lbs|pounds?)',
    }

    pattern = patterns.get(entity.lower())
    if pattern:
        match = re.search(pattern, text.lower())
        if match:
            return f"{match.group(1)} {entity.lower()}", match.group(1)
    
    return f"{entity} not found", None

# Function to process a single row of the DataFrame
def process_row(row, image_directory):
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Download the image if it's a URL
    if image_url.startswith('http'):
        image_path = download_image(image_url, image_directory)
    else:
        image_path = os.path.join(image_directory, image_url)

    if not image_path:
        return "Download Error", "", None

    # Extract text without rotation
    final_text = extract_text_from_image(image_path)

    if not final_text:
        return "Image Error", "", None

    # Search for the entity in the extracted text
    entity_result, entity_value = search_entity(final_text, entity_name)

    return entity_result, final_text, entity_value

# Directory to save downloaded images
image_directory = '/kaggle/working/downloaded_images/'

# Create directory if it doesn't exist
if not os.path.exists(image_directory):
    os.makedirs(image_directory)

# Load the CSV containing image URLs and entity names
csv_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_path)

# Process a limited number of rows from the DataFrame
limit = 1000  # Set limit on the number of images to process
df_limited = df.head(limit)

# Process the rows in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: process_row(row, image_directory), [row for _, row in df_limited.iterrows()]))

# Unpack results
predictions, extracted_texts, entity_values = zip(*results)

# Add the predictions, extracted texts, and entity values to the DataFrame
df_limited['extracted_text'] = extracted_texts
df_limited['prediction'] = predictions
df_limited['entity_value'] = entity_values

# Save the updated DataFrame to a CSV file
output_csv_path = '/kaggle/working/entity_predictions_with_text_and_value.csv'
df_limited.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")
"""

# Text extraction for testing

In [ ]:
"""
import easyocr
from PIL import Image
import re
import os
import pandas as pd
import requests
from io import BytesIO
import concurrent.futures
import numpy as np

# Initialize the EasyOCR reader (using GPU if available)
reader = easyocr.Reader(['en'], gpu=True)

# Function to download an image or load it from the local directory
def download_image(image_url, save_directory):
    try:
        response = requests.get(image_url, timeout=5)
        response.raise_for_status()  # Ensure we notice bad responses
        img = Image.open(BytesIO(response.content))
        image_name = os.path.basename(image_url)
        save_path = os.path.join(save_directory, image_name)
        img.save(save_path)
        return save_path
    except Exception as e:
        print(f"Error downloading image {image_url}: {e}")
        return None

# Function to extract text from an image without rotation
def extract_text_from_image(image_path):
    try:
        img = Image.open(image_path)

        # Resize image to a smaller size for faster processing (reduce by half)
        img = img.resize((img.width // 2, img.height // 2))

        # Convert image to NumPy array for OCR
        img_np = np.array(img)

        # Extract text using OCR
        img_text = reader.readtext(img_np)
        final_text = " ".join([text for _, text, __ in img_text])

        return final_text
    except OSError as e:
        print(f"Error processing image {image_path}: {e}")
        return ""

# Function to search for a specific entity using regular expressions
def search_entity(text, entity):
    patterns = {
        'grams': r'(\d+(\.\d+)?)\s*(g|grams?)',
        'milligrams': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
        'kilograms': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
        'ounces': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
        'newtons': r'(\d+(\.\d+)?)\s*(n|newtons?)',
        'liters': r'(\d+(\.\d+)?)\s*(l|liters?|ml|milliliters?)',
        'watts': r'(\d+(\.\d+)?)\s*(w|watts?)',
        'volts': r'(\d+(\.\d+)?)\s*(v|volts?)',
        'inches': r'(\d+(\.\d+)?)\s*(in|inches?)',
        'centimeters': r'(\d+(\.\d+)?)\s*(cm|centimeters?)',
        'meters': r'(\d+(\.\d+)?)\s*(m|meters?)',
        'feet': r'(\d+(\.\d+)?)\s*(ft|feet?)',
        'pounds': r'(\d+(\.\d+)?)\s*(lbs|pounds?)',
    }

    pattern = patterns.get(entity.lower())
    if pattern:
        match = re.search(pattern, text.lower())
        if match:
            return f"{match.group(1)} {entity.lower()}", match.group(1)
    
    return f"{entity} not found", None

# Function to process a single row of the DataFrame
def process_row(row, image_directory):
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Download the image if it's a URL
    if image_url.startswith('http'):
        image_path = download_image(image_url, image_directory)
    else:
        image_path = os.path.join(image_directory, image_url)

    if not image_path or not os.path.isfile(image_path):
        return "Download Error", "", None

    # Extract text without rotation
    final_text = extract_text_from_image(image_path)

    if not final_text:
        return "Image Error", "", None

    # Search for the entity in the extracted text
    entity_result, entity_value = search_entity(final_text, entity_name)

    return entity_result, final_text, entity_value

# Directory to save downloaded images
image_directory = '/kaggle/working/downloaded_test_images/'

# Create directory if it doesn't exist
if not os.path.exists(image_directory):
    os.makedirs(image_directory)

# Load the CSV containing image URLs and entity names
csv_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/test.csv'
df = pd.read_csv(csv_path)

# Process a limited number of rows from the DataFrame
limit = 1000  # Set limit on the number of images to process
df_limited = df.head(limit)

# Process the rows in parallel using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: process_row(row, image_directory), [row for _, row in df_limited.iterrows()]))

# Unpack results
predictions, extracted_texts, entity_values = zip(*results)

# Add the predictions, extracted texts, and entity values to the DataFrame using .loc
df_limited.loc[:, 'extracted_text'] = extracted_texts
df_limited.loc[:, 'prediction'] = predictions
df_limited.loc[:, 'entity_value'] = entity_values

# Save the updated DataFrame to a CSV file
output_csv_path = '/kaggle/working/test_entity_predictions_with_text_and_value.csv'
df_limited.to_csv(output_csv_path, index=False)

print(f"Results saved to {output_csv_path}")
"""

In [ ]:
#train = pd.read_csv('entity_predictions_with_text_and_value.csv')

In [ ]:
#train = train[['group_id', 'image_link', 'entity_name', 'extracted_text']]

In [ ]:
#train.head()

In [ ]:
#test = pd.read_csv('test_entity_predictions_with_text_and_value.csv')

In [ ]:
#test.head(100)

In [ ]:
"""# Extract the 4th row of a specific column (e.g., 'Column_Name')
column_name = 'extracted_text'
fourth_row_value = test.at[4, column_name]  # 3 is the index for the 4th row (0-based indexing)

print(f"The 4th row value of '{column_name}' is: {fourth_row_value}")
"""

In [ ]:
#test1 = test[['group_id', 'image_link', 'entity_name', 'extracted_text', 'entity_value']]

In [ ]:
#test1.head()

In [ ]:
#train1 = train[['group_id', 'image_link', 'entity_name', 'extracted_text']]

In [ ]:
#train1.head()

In [ ]:
#train1.shape

In [ ]:
#df1 = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [ ]:
#df1

# merge

In [ ]:
# Merging df1 and test1 on image_link, entity_name, and group_id
#df1 = df1.merge(train1, on=['image_link', 'entity_name', 'group_id'], how='inner')

# Optionally, if you need to remove duplicates based on these columns:
#df1 = df1.drop_duplicates(subset=['image_link', 'entity_name', 'group_id'])


In [ ]:
#df1

In [ ]:
# Display the 5th value of the 'extracted_text' column
#value = df1['extracted_text'].iloc[4]
#print("The 5th value in the 'extracted_text' column is:", value)


# Probability

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate the prediction and the matching score
def predict_presence(entity_value, predicted, threshold=70):
    # Calculate the fuzzy match ratio between the entity and the extracted text
    match_score = fuzz.partial_ratio(entity_value.lower(), extracted_text.lower())
    
    # Predict presence based on the match score exceeding the threshold
    presence = 1 if match_score >= threshold else 0
    
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply the function to each row in the DataFrame
pred[['prediction', 'probability']] = pred.apply(
    lambda row: predict_presence(row['entity_value'], row['extracted_text']), axis=1, result_type='expand'
)



In [ ]:
# Display the cleaned DataFrame
pred.head(30)

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Function to calculate the prediction and the matching score
def predict_presence(entity_value, extracted_text, threshold=70):
    # Calculate the fuzzy match ratio between the entity and the extracted text
    match_score = fuzz.partial_ratio(entity_value.lower(), extracted_text.lower())
    
    # Predict presence based on the match score exceeding the threshold
    presence = 1 if match_score >= threshold else 0
    
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply the function to each row in the DataFrame
df1[['prediction', 'probability']] = df1.apply(
    lambda row: predict_presence(row['entity_value'], row['extracted_text']), axis=1, result_type='expand'
)

# Show the DataFrame with predictions and probabilities
print(df1)


In [ ]:
"""
import pandas as pd
import re

# Define the patterns for different units (case insensitive)
patterns = {
    'grams': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'milligrams': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'kilograms': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'ounces': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'newtons': r'(\d+(\.\d+)?)\s*(n|newtons?)',
    'liters': r'(\d+(\.\d+)?)\s*(l|liters?|ml|milliliters?)',
    'watts': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'volts': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'inches': r'(\d+(\.\d+)?)\s*(in|inches?)',
    'centimeters': r'(\d+(\.\d+)?)\s*(cm|centimeters?)',
    'meters': r'(\d+(\.\d+)?)\s*(m|meters?)',
    'feet': r'(\d+(\.\d+)?)\s*(ft|feet?)',
    'pounds': r'(\d+(\.\d+)?)\s*(lbs|pounds?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'weight': ['grams', 'milligrams', 'kilograms', 'ounces', 'pounds'],
    'volume': ['liters', 'milliliters'],
    'force': ['newtons'],
    'electric_power': ['watts'],
    'voltage': ['volts'],
    'length': ['inches', 'centimeters', 'meters', 'feet']
}

# Function to clean extracted text and remove noise
def clean_extracted_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and keep alphanumeric and spaces
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Function to find the value with the appropriate unit in extracted_text based on the entity name
def find_unit_in_text(entity_name, extracted_text):
    # Ensure extracted_text is a string
    if pd.isna(extracted_text):
        return None  # If no extracted text, return None
    
    # Clean the extracted text by removing noise
    extracted_text = clean_extracted_text(str(extracted_text))

    # Determine the type of entity (weight, volume, etc.)
    entity_type = None
    for key in entity_unit_map:
        if key in entity_name.lower():
            entity_type = key
            break
    
    if entity_type is None:
        return None  # If entity type is not recognized, return None
    
    # Search for numeric value followed by a unit based on entity type
    for unit_name in entity_unit_map[entity_type]:
        pattern = patterns[unit_name]  # Get the regex pattern for this unit
        match = re.search(pattern, extracted_text, re.IGNORECASE)
        if match:
            return match.group(0)  # Return the matched value with the unit
    
    # If no match is found, return None
    return None

# Apply the function to each row in the DataFrame
df1['value_with_unit'] = df1.apply(
    lambda row: find_unit_in_text(row['entity_value'], row['extracted_text']), axis=1
)

# Show the DataFrame with the extracted value with units
df1 = df1[['image_link', 'entity_value', 'extracted_text', 'value_with_unit']]

# Save the updated DataFrame to a new CSV file
df1.to_csv('updated_values_with_units.csv', index=False)

# Show the updated DataFrame
print(df1)
"""

In [ ]:
#df1.head(100)

# pytesseract

In [ ]:
#pip install doctr

In [ ]:
"""
import easyocr
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import numpy as np
import re

# Function to rotate an image
def rotate_image(image_path, angle):
    with Image.open(image_path) as img:
        rotated_img = img.rotate(angle, expand=True)
        return rotated_img

# Function to preprocess the image
def preprocess_image(image):
    # Convert to grayscale
    image = image.convert('L')
    # Apply adaptive thresholding
    image = ImageOps.autocontrast(image)
    # Enhance the image
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)  # Increase contrast
    # Apply a filter
    image = image.filter(ImageFilter.SHARPEN)
    return image

# Function to perform OCR and check extracted text
def perform_ocr(image):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(np.array(image))
    return result

# Function to clean extracted text
def clean_text(text):
    # Remove unwanted characters and normalize spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove non-alphanumeric characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

# Function to evaluate if the extracted text is "good"
def is_text_good(result):
    # Define criteria for good text (e.g., non-empty, specific content)
    return any(len(clean_text(detection[1])) > 5 for detection in result)

# Main function to handle rotating and OCR
def process_image(image_path):
    angles = [0, 90, 180, 270]  # Angles to rotate the image
    for angle in angles:
        rotated_image = rotate_image(image_path, angle)
        preprocessed_image = preprocess_image(rotated_image)
        result = perform_ocr(preprocessed_image)
        
        if is_text_good(result):
            return result, angle  # Return the successful result and angle
    
    return None, None  # No satisfactory result found

# Path to your image
image_path = '/kaggle/working/downloaded_images/71u9qpY7TtL.jpg'

# Process the image
result, angle = process_image(image_path)

if result:
    print(f"Text extracted from image rotated by {angle} degrees:")
    for detection in result:
        cleaned_text = clean_text(detection[1])
        print(cleaned_text)
else:
    print("No satisfactory text found.")
"""

In [ ]:
#df1.head(100)

In [ ]:
# Display the 5th value of the 'extracted_text' column
#value = df1['extracted_text'].iloc[97]
#print("The 5th value in the 'extracted_text' column is:", value)


# Training

In [20]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# Function to remove units from text (keeping only numeric values)
def remove_units(text):
    return re.sub(r'[^0-9.]', '', str(text))

# Function to calculate the prediction and matching score based on text
def predict_presence(entity_value, extracted_text, threshold=40):
    clean_entity_value = remove_units(entity_value)
    clean_extracted_text = remove_units(extracted_text)
    match_score = fuzz.partial_ratio(clean_entity_value, clean_extracted_text)
    presence = 1 if match_score >= threshold else 0
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply transformations to the images for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading both image and text data
class CustomDataset(Dataset):
    def __init__(self, pred, transform=None):
        self.pred = pred
        self.transform = transform
    
    def __len__(self):
        return len(self.pred)
    
    def __getitem__(self, idx):
        # Load image
        img_path = self.pred.iloc[idx]['image_link']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        # Get text features
        entity_value = self.pred.iloc[idx]['entity_value']
        extracted_text = self.pred.iloc[idx]['predicted']
        prediction, probability = predict_presence(entity_value, extracted_text)
        
        # Target label: prediction presence
        target = torch.tensor(prediction, dtype=torch.float32)
        
        return image, torch.tensor([probability], dtype=torch.float32), target

# Create the dataset and dataloader
dataset = CustomDataset(df1, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove the last fully connected layer

# Classifier to predict presence based on both image features and text-based probability
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.resnet = resnet
        self.fc = nn.Linear(512 + 1, 1)  # 512 from ResNet features + 1 for text-based probability
        
    def forward(self, image, text_prob):
        image_features = self.resnet(image)  # Extract image features using ResNet
        combined_features = torch.cat((image_features, text_prob), dim=1)  # Concatenate with text probability
        output = self.fc(combined_features)  # Final prediction
        return output

# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'df1' is not defined

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import requests
from io import BytesIO
from sklearn.metrics import accuracy_score, f1_score

# Function to remove units from text (keeping only numeric values)
def remove_units(text):
    return re.sub(r'[^0-9.]', '', str(text))

# Function to calculate the prediction and matching score based on text
def predict_presence(entity_value, extracted_text, threshold=40):
    clean_entity_value = remove_units(entity_value)
    clean_extracted_text = remove_units(extracted_text)
    match_score = fuzz.partial_ratio(clean_entity_value, clean_extracted_text)
    presence = 1 if match_score >= threshold else 0
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply transformations to the images for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading both image and text data
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image from URL
        img_url = self.df.iloc[idx]['image_link']
        
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error loading image from {img_url}: {e}")
            # Return a dummy tensor if there's an error
            image = Image.new('RGB', (224, 224), color=(255, 255, 255))

        if self.transform:
            image = self.transform(image)
        
        # Get text features
        entity_value = self.df.iloc[idx]['entity_value']
        extracted_text = self.df.iloc[idx]['extracted_text']
        prediction, probability = predict_presence(entity_value, extracted_text)
        
        # Target label: prediction presence
        target = torch.tensor(prediction, dtype=torch.float32)
        
        return image, torch.tensor([probability], dtype=torch.float32), target

# Load your dataset (ensure df1 is defined)
df1 = pd.read_csv('/path/to/your/dataset.csv')  # Adjust path as necessary
dataset = CustomDataset(df1, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove the last fully connected layer

# Classifier to predict presence based on both image features and text-based probability
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.resnet = resnet
        self.fc = nn.Linear(512 + 1, 1)  # 512 from ResNet features + 1 for text-based probability
        
    def forward(self, image, text_prob):
        image_features = self.resnet(image)  # Extract image features using ResNet
        combined_features = torch.cat((image_features, text_prob), dim=1)  # Concatenate with text probability
        output = self.fc(combined_features)  # Final prediction
        return output

# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate predictions and apply sigmoid for probability thresholding
def calculate_predictions(outputs, threshold=0.5):
    return (torch.sigmoid(outputs) >= threshold).float()

# Training loop with accuracy and F1 score calculation
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        all_preds = []
        all_targets = []
        
        for images, text_probs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(images, text_probs).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Get predictions and store them
            preds = calculate_predictions(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
        
        # Calculate accuracy and F1 score after each epoch
        acc = accuracy_score(all_targets, all_preds)
        f1 = f1_score(all_targets, all_preds)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}, Accuracy: {acc}, F1 Score: {f1}')

# Train the model
train_model(model, dataloader, criterion, optimizer, num_epochs=10)


In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import requests
from io import BytesIO
from sklearn.metrics import accuracy_score, f1_score

# Function to remove non-alphanumeric characters except units
def clean_text(text):
    return re.sub(r'[^\w\s.%]', '', str(text))  # Keep only words, spaces, and common units like % and .

# Function to calculate the prediction based on text comparison, return matched value with unit if present
def predict_presence(entity_value, extracted_text, threshold=40):
    clean_entity_value = clean_text(entity_value)
    clean_extracted_text = clean_text(extracted_text)
    
    match_score = fuzz.partial_ratio(clean_entity_value, clean_extracted_text)
    if match_score >= threshold:
        # Return the part of the extracted text that matches the entity value along with its unit
        return clean_entity_value, match_score / 100
    else:
        # No match found, return an empty string and match score 0
        return "", 0

# Apply transformations to the images for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading both image and text data
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image from URL
        img_url = self.df.iloc[idx]['image_link']
        
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error loading image from {img_url}: {e}")
            # Return a dummy tensor if there's an error
            image = Image.new('RGB', (224, 224), color=(255, 255, 255))

        if self.transform:
            image = self.transform(image)
        
        # Get text features
        entity_value = self.df.iloc[idx]['entity_value']
        extracted_text = self.df.iloc[idx]['extracted_text']
        matched_value, probability = predict_presence(entity_value, extracted_text)
        
        # If matched_value is empty, return 'no match'
        if matched_value == "":
            matched_value = " "
        
        return image, torch.tensor([probability], dtype=torch.float32), matched_value

# Create the dataset and dataloader
dataset = CustomDataset(df1, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove the last fully connected layer

# Classifier to predict presence based on both image features and text-based probability
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.resnet = resnet
        self.fc = nn.Linear(512 + 1, 1)  # 512 from ResNet features + 1 for text-based probability
        
    def forward(self, image, text_prob):
        image_features = self.resnet(image)  # Extract image features using ResNet
        combined_features = torch.cat((image_features, text_prob), dim=1)  # Concatenate with text probability
        output = self.fc(combined_features)  # Final prediction
        return output

# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate predictions and apply sigmoid for probability thresholding
def calculate_predictions(outputs, threshold=0.5):
    return (torch.sigmoid(outputs) >= threshold).float()

# Training loop with accuracy and F1 score calculation
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        all_preds = []
        all_targets = []
        all_matches = []
        
        for images, text_probs, matched_values in dataloader:
            optimizer.zero_grad()
            outputs = model(images, text_probs).squeeze()
            # Dummy targets, as we're more interested in storing matched values
            targets = torch.zeros(len(outputs))  # Since we're using BCEWithLogitsLoss, the target must be provided
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Store matched values
            all_matches.extend(matched_values)

        # Print or log the results after each epoch
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}')
        
        # Display the matched values
        print(f'Matched values: {all_matches[:5]}...')  # Show the first 5 matched values for brevity

# Train the model
train_model(model, dataloader, criterion, optimizer, num_epochs=10)


In [ ]:
df1

In [ ]:
test1.head()

In [ ]:
# Merging df1 and test1 on image_link, entity_name, and group_id
df1 = df1.merge(test1, on=['image_link', 'extracted_text'], how='inner')

# Optionally, if you need to remove duplicates based on these columns:
#df1 = df1.drop_duplicates(subset=['image_link', 'entity_name', 'group_id'])

In [ ]:
df1.head()

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from fuzzywuzzy import fuzz
import re
import requests
from PIL import Image
from io import BytesIO
from torchvision import transforms

# Function to clean text
def clean_text(text):
    return re.sub(r'[^\w\s.%]', '', str(text))  # Keep only alphanumeric characters, spaces, and common units

# Function to predict the best entity value based on the entity name
def predict_entity_value(entity_name, extracted_text, threshold=40):
    clean_extracted_text = clean_text(extracted_text)
    
    # Compare extracted text with entity_name and extract the most probable entity value
    match_score = fuzz.partial_ratio(clean_text(entity_name), clean_extracted_text)
    
    if match_score >= threshold:
        return clean_extracted_text, match_score / 100  # Return extracted text as predicted value and probability
    else:
        return "", 0  # No match found

# Custom dataset class for test data
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image from URL
        img_url = self.df.iloc[idx]['image_link']
        
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error loading image from {img_url}: {e}")
            image = Image.new('RGB', (224, 224), color=(255, 255, 255))  # Dummy image if error occurs

        if self.transform:
            image = self.transform(image)
        
        # Extract entity name and extracted text
        entity_name = self.df.iloc[idx]['entity_name']
        extracted_text = self.df.iloc[idx]['extracted_text']
        
        # Predict the entity value based on extracted text
        matched_value, probability = predict_entity_value(entity_name, extracted_text)
        
        if matched_value == "":
            matched_value = "no match"
        
        return image, torch.tensor([probability], dtype=torch.float32), matched_value

# Create the test dataloader
def get_test_dataloader(test_df, transform, batch_size=32):
    test_dataset = TestDataset(test_df, transform=transform)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_dataloader

# Testing function to make predictions and format them
def test_model(model, test_dataloader):
    model.eval()
    unit_predictions = []

    with torch.no_grad():
        for images, text_probs, matched_values in test_dataloader:
            outputs = model(images, text_probs).squeeze()

            # Collect and format predictions
            for i in range(len(matched_values)):
                matched_value = matched_values[i].strip()
                if matched_value != "no match":
                    prediction = f"{float(text_probs[i].item()):.2f} {matched_value}"
                    unit_predictions.append(prediction)
                else:
                    unit_predictions.append("no match")

    return unit_predictions

# Testing the model on a new CSV file
def test_on_csv(model, test_csv_path, output_csv_path):
    # Load the test CSV
    test_df = df1

    # Create the test dataloader
    test_dataloader = get_test_dataloader(test_df, transform)

    # Get predictions in the required format
    unit_predictions = test_model(model, test_dataloader)

    # Assign predictions to the DataFrame
    test_df['predicted_value'] = unit_predictions

    # Save the predictions to a new CSV file
    test_df.to_csv(output_csv_path, index=False)

    print("Test predictions saved to:", output_csv_path)
    print(test_df[['entity_name', 'extracted_text', 'predicted_value']].head())

# Assuming the test CSV is located at the following path
test_csv_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/test.csv'
output_csv_path = 'test_predictions_with_entities.csv'

# Run the testing on the CSV
test_on_csv(model, test_csv_path, output_csv_path)

In [ ]:
TR = pd.read_csv('/kaggle/working/test_predictions_with_entities.csv')

In [ ]:
TR.head(100)

# For test dataset

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import requests
from io import BytesIO
from sklearn.metrics import accuracy_score, f1_score

# Function to remove units from text (keeping only numeric values)
def remove_units(text):
    return re.sub(r'[^0-9.]', '', str(text))

# Function to calculate the prediction and matching score based on text
def predict_presence(entity_value, extracted_text, threshold=50):
    clean_entity_value = remove_units(entity_value)
    clean_extracted_text = remove_units(extracted_text)
    match_score = fuzz.partial_ratio(clean_entity_value, clean_extracted_text)
    presence = 1 if match_score >= threshold else 0
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply transformations to the images for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading both image and text data
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image from URL
        img_url = self.df.iloc[idx]['image_link']
        
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error loading image from {img_url}: {e}")
            # Return a dummy tensor if there's an error
            image = Image.new('RGB', (224, 224), color=(255, 255, 255))
        
        if self.transform:
            image = self.transform(image)
        
        # Get text features
        entity_value = self.df.iloc[idx].get('entity_value', '0')
        extracted_text = self.df.iloc[idx].get('extracted_text', '')
        prediction, probability = predict_presence(entity_value, extracted_text)
        
        # Ensure targets are converted to floats
        target = torch.tensor(float(prediction), dtype=torch.float32)
        text_prob = torch.tensor(float(probability), dtype=torch.float32).unsqueeze(0)  # Ensure it's a 2D tensor
        
        return image, text_prob, target, img_url, extracted_text

# Load pre-trained ResNet model
resnet = models.resnet18(weights='DEFAULT')  # Use the new weights argument
resnet.fc = nn.Identity()  # Remove the last fully connected layer

# Classifier to predict presence based on both image features and text-based probability
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__() 
        self.resnet = resnet
        self.fc = nn.Linear(512 + 1, 1)  # 512 from ResNet features + 1 for text-based probability
        
    def forward(self, image, text_prob):
        image_features = self.resnet(image)  # Extract image features using ResNet
        combined_features = torch.cat((image_features, text_prob), dim=1)  # Concatenate with text probability
        output = self.fc(combined_features)  # Final prediction
        return output

# Function to calculate predictions and apply sigmoid for probability thresholding
def calculate_predictions(outputs, threshold=0.5):
    return (torch.sigmoid(outputs) >= threshold).float()

# Function to train the model
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    max_accuracy = 0
    max_f1 = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        # Store predictions and targets for calculating metrics later
        all_preds = []
        all_targets = []
        
        for images, text_probs, targets, _, _ in dataloader:
            optimizer.zero_grad()
            outputs = model(images, text_probs).squeeze()
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            # Calculate predictions and store
            preds = calculate_predictions(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
        
        # Calculate accuracy and F1 score for this epoch
        epoch_accuracy = accuracy_score(all_targets, all_preds)
        epoch_f1 = f1_score(all_targets, all_preds)
        
        # Update max values if current epoch scores are better
        if epoch_accuracy > max_accuracy:
            max_accuracy = epoch_accuracy
        if epoch_f1 > max_f1:
            max_f1 = epoch_f1
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}, Accuracy: {epoch_accuracy}, F1 Score: {epoch_f1}')
    
    print(f'Max Accuracy: {max_accuracy}, Max F1 Score: {max_f1}')

# Function to test the model on the test data and return predictions
def test_model_with_predictions(model, dataloader):
    model.eval()
    all_preds = []
    all_targets = []
    all_image_links = []
    all_extracted_texts = []
    
    with torch.no_grad():
        for images, text_probs, targets, img_urls, extracted_texts in dataloader:
            outputs = model(images, text_probs).squeeze()
            preds = calculate_predictions(outputs)
            
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_image_links.extend(img_urls)
            all_extracted_texts.extend(extracted_texts)
    
    # Convert predictions to a DataFrame for easy viewing
    results_df = pd.DataFrame({
        'image_link': all_image_links,
        'extracted_text': all_extracted_texts,
        'true_target': all_targets,
        'predicted_presence': all_preds
    })
    
    # Calculate accuracy and F1 score on test set
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)
    
    print(f'Test Accuracy: {acc}, Test F1 Score: {f1}')
    return results_df, acc, f1

# Load training data (assuming df1 is your training dataframe)
dataset_train = CustomDataset(df1, transform=transform)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

# Load test data (assuming df_test is your test dataframe)
df_test = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/test.csv')  # Load your test dataset here
dataset_test = CustomDataset(df_test, transform=transform)
dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)

# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, dataloader_train, criterion, optimizer, num_epochs=10)

# Test the model on the test dataset and get predictions
test_results_df, test_accuracy, test_f1 = test_model_with_predictions(model, dataloader_test)

# Display the predictions (you can also save this to a CSV file)
print(test_results_df.head())

# To save to CSV
test_results_df.to_csv('test_predictions.csv', index=False)


In [ ]:

# Check the column names
df1.head()


# TESTING 

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import requests
from io import BytesIO
from sklearn.metrics import accuracy_score, f1_score

# Function to remove units from text (keeping only numeric values)
def remove_units(text):
    return re.sub(r'[^0-9.]', '', str(text))

# Function to calculate the prediction and matching score based on text
def predict_presence(entity_value, extracted_text, threshold=40):
    clean_entity_value = remove_units(entity_value)
    clean_extracted_text = remove_units(extracted_text)
    match_score = fuzz.partial_ratio(clean_entity_value, clean_extracted_text)
    presence = 1 if match_score >= threshold else 0
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Apply transformations to the images for ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class for loading both image and text data
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # Load image from URL
        img_url = self.df.iloc[idx]['image_link']
        
        try:
            response = requests.get(img_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
        except Exception as e:
            print(f"Error loading image from {img_url}: {e}")
            # Return a dummy tensor if there's an error
            image = Image.new('RGB', (224, 224), color=(255, 255, 255))

        if self.transform:
            image = self.transform(image)
        
        # Get text features
        entity_value = self.df.iloc[idx]['entity_value']
        extracted_text = self.df.iloc[idx]['extracted_text']
        prediction, probability = predict_presence(entity_value, extracted_text)
        
        # Target label: prediction presence
        target = torch.tensor(prediction, dtype=torch.float32)
        
        return image, torch.tensor([probability], dtype=torch.float32), target

# Load pre-trained ResNet model
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # Remove the last fully connected layer

# Classifier to predict presence based on both image features and text-based probability
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.resnet = resnet
        self.fc = nn.Linear(512 + 1, 1)  # 512 from ResNet features + 1 for text-based probability
        
    def forward(self, image, text_prob):
        image_features = self.resnet(image)  # Extract image features using ResNet
        combined_features = torch.cat((image_features, text_prob), dim=1)  # Concatenate with text probability
        output = self.fc(combined_features)  # Final prediction
        return output

# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.BCEWithLogitsLoss()  # Binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate predictions and apply sigmoid for probability thresholding
def calculate_predictions(outputs, threshold=0.5):
    return (torch.sigmoid(outputs) >= threshold).float()

# Training loop with accuracy and F1 score calculation
def train_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        all_preds = []
        all_targets = []
        
        for images, text_probs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(images, text_probs).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Get predictions and store them
            preds = calculate_predictions(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
        
        # Calculate accuracy and F1 score after each epoch
        acc = accuracy_score(all_targets, all_preds)
        f1 = f1_score(all_targets, all_preds)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}, Accuracy: {acc}, F1 Score: {f1}')

# Train the model
train_model(model, dataloader, criterion, optimizer, num_epochs=10)




In [ ]:
# Create testing dataset and dataloader
def create_test_dataloader(test_df, batch_size=32):
    test_dataset = CustomDataset(test_df, transform=transform)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return test_dataloader

# Predict on test data
def predict_on_test_data(model, test_dataloader):
    model.eval()
    all_predictions = []
    with torch.no_grad():
        for images, text_probs, _ in test_dataloader:
            outputs = model(images, text_probs).squeeze()
            preds = calculate_predictions(outputs)
            all_predictions.extend(preds.cpu().numpy())
    return all_predictions

# Example test DataFrame
test_df = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/test.csv')  # Assuming 'pred' contains the testing data as well

# Create the test dataloader
test_dataloader = create_test_dataloader(test_df)

# Predict on test data
predictions = predict_on_test_data(model, test_dataloader)

# Adding predictions to the test DataFrame
test_df['predictions'] = predictions


In [ ]:
test_df[['entity_name', 'predictions']]